# Hands-on session Thursday 16
# Photometric redshift using Decision Trees, Random Forest and fully connected neural network

This notebook is designed to guide you step by step through the development of the mini-project by following each cell. In some cases, **specific tasks are highlighted in bold** for you to complete. You can review the provided solutions to compare your results.



### What is redshift?

We now know that stars and galaxies are located at varying distances, sometimes billions of light-years away. Measuring these distances is crucial for creating a 3D map of the Universe and converting apparent brightness into intrinsic luminosity, revealing the true energy output of astrophysical sources. Additionally, because light travels at a finite speed, distance corresponds to lookback time, allowing us to study the Universe’s history.

Moreover, Edwin Hubble discovered in 1929 that the universe is *expanding*, and the farther away a galaxy is, the faster it is moving away from us. This causes a Doppler effect in the light we receive from far objects. In astrophysics, this doppler shift is known as **redshift** $z$, and can be expressed in respect to the observed and emitted wavelength:

$$ 1 + z =\frac{\lambda_{obs}}{\lambda_{em}} $$

The best way to determine redshifts is by calculating the shift of known lines in a *spectrum*, which is the light emission as a function of wavelength.  However, spectra might be low quality if the objects aret too far away, and is expensive to obtain high quality spectral data. Photometry, on the other hand, is much cheaper and easy to obtain. In this case, we have the average emission (or brightness) in a range of wavelengths (known as band, or filter).



<img src="https://skyserver.sdss.org/dr14/en/get/SpecById.ashx?id=1833056915844786176" alt="spectra" width="500">



Therefore, we can avail ourselves of photometric data for much larger samples of galaxies in the Universe, and upcoming surveys are slated to image a significant fraction (>10%) of all the galaxies in the Universe. Being able to derive reliable redshifts for these galaxies is crucial for astronomy.







### The dataset
The dataset used in this notebook comes from this paper by [Zhou et. al (2019)](https://academic.oup.com/mnras/article/488/4/4565/5538813). It is a compilation from several surveys, such as DEEP2, DEEP3 and 3D-HST. The data resembles the wavelength coverage and depth of the upcoming Vera Rubin Observatory which is expected to provide photometry in six bands, ranging from near- ultraviolet to near-infrared (u, g, r, i, z, and y), of approximately 20 billion galaxies, spanning a considerable fraction of the Universe’s volume.


The results obtained in the paper are shown in the following figure


<img src="https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/mnras/488/4/10.1093_mnras_stz1866/1/stz1866fig5.jpeg?Expires=1739848857&Signature=A1JoKdBTtx6umURSd7FPeRWR19vwiRfeiOyK7WSoo-ZTE6-B5QXwy7HCyPmiUBykW8d1e60lQ4CFLvcHjinABBnbfl7xHMgMvx9gHGRIgISI6j-8bxfCaWGojXMhdU-ZkUxwhJ0OsbIGrYDCp1W~HECl10BOVZ3HPq7Qjv6D4BPYsSMX-JGqNoqW-lIxJ0HxWhxl63kPGvfwvdt~H1ZL7KaK2oDaPMhepxqRvr17P-~H7TW0j-lY-D940m5il-Ffq0Bs17Rf6sUDIj3qwwvvU~km~11W4Zrf2RYf~rDf1w3gXY2zAvYojYwdOIFo3Mo3wR6hg7RGuOrmrMDz8E7PAQ__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA" alt="photo_z_paper" width="500">


where  $\sigma_{NMAD}$ is the normalized median absolute deviation of the residuals and $\eta$ is the outlier fraction defined as the objects with redshifts such as $\frac{|(z_{spec} - z)|}{(1+z_{spec})} > 0.15$.

 In the paper, the values are $\sigma_{NMAD}=0.0534$ and $\eta=10.61\%$ (outlier fraction)


**You can explore the data and check the available features in the data set. Check the columns and the number of objects present in the database**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split, RandomizedSearchCV, validation_curve
from sklearn.model_selection import KFold, cross_validate, cross_val_predict, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 150)

font = {'size'   : 10}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=8)
matplotlib.rc('ytick', labelsize=8)
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
!pip install astropy



In [ ]:
import astropy

from astropy.io import fits

In [ ]:
with fits.open('/content/drive/MyDrive/escuela_ML_USM_2025/DEEP2_uniq_Terapix_Subaru_v1.fits') as data:
    df = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder())
    #https://numpy.org/doc/2.1/user/byteswapping.html

In [ ]:
#explore the data

**From the features, we need to select only the brightness of the galaxies, in the filters of interest ('u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor').**

<img src="https://noirlab.edu/science/sites/default/files/media/archives/images/filters_u_DR1.png" alt="ugrizY" width="500">


**Filter the dataframe to only have those features as columns in a dataframe named `features`**

**The variable we want to predict (target) is the redshift, `zhelio`; define an array or Series `target` with the values of redshift**

### Part 1: Decision Trees

First we will implement a simple decision tree (regressor) to predict the redshift. You need to split the data into train and test, select the ratio as you wish (always leave the bigger fraction for training!)


In [ ]:
#split te data into train/validation/test

indices = np.arange(len(target))

# first reserve 70% of the data for training, 30% for validation
X_train, X_validate, y_train, y_validate, indices_train, indices_validate = train_test_split(features, target, indices, test_size=0.3, random_state=42)

# second, split the validation set in half to obtain validation and test sets.
X_validate, X_test, y_validate, y_test, indices_validate, indices_test = train_test_split(X_validate, y_validate, indices_validate, test_size=0.5, random_state=42)

In [ ]:
print(X_train.shape)
print(X_validate.shape)
print(X_test.shape)

In [ ]:
print(y_test.shape)
print(y_validate.shape)
print(y_train.shape)


A decision tree is composed of a series of if-else decision steps. The number of steps and the types of decision at each step is determined by training the algorithm with supervision.
Call DecisionTreeRegressor (with the default parameteres....for now) from sklearn and fit the tree to your training data. Then, obtain the predictions in your test sets

In [ ]:
# Initialize the model
dtree = DecisionTreeRegressor()

# Fit the model parameters using the training dataset
dtree.fit(X_train, y_train)

In [ ]:
y_predict = dtree.predict(X_test)


In [ ]:
print(y_test.shape)

**Since this is a Regression problem, obtain a proper metric to evaluate this algorithm (MAE, MAPE, MSE, etc), and calculate the NMAD and outlier fraction. Also, plot the distribution of redshifts (in the range 0-3) for the predicted values and the true (test) values**

In [ ]:
print(f'MSE = {mean_squared_error(y_test, y_predict):.4f}')

In [ ]:
plt.hist(y_test,bins=50,density=False,alpha=0.5, range = (0,3), label = 'True');
plt.hist(y_predict,bins=50,density=False,alpha=0.5, range = (0,3), color = 'g', label = 'Predicted');
plt.legend(fontsize=14);

**To calculate NMAD and outlier fraction, maybe you need to flatten the arrays, see line below:**

In [ ]:
y_test = np.array(y_test).flatten()  # Flatten if needed
y_predict = np.array(y_predict).flatten()  # Flatten if needed

In [ ]:
print(y_test.size)
print(y_predict.size)

In [ ]:
1.48*np.median(np.abs(y_test-y_predict)/(1 + y_test)) #NMAD


In [ ]:
len(np.where(np.abs(y_test-y_predict)>0.15*(1+y_test))[0])/len(y_test) #outlier fraction

**How does your result compares with the results in the paper?**

**Make a plot of Truth (test) and Prediction to check how they compare**

In [ ]:
#plot here


**What do you think of your results?**

Not so good!


One way to improve the results of a model is to tune its parameters to run appropiately. In decision trees, this is particularly important, since an unleashed tree will try to fit all the train data, which can lead to overfitting.

Below is an implementation of [`RandomizedSearchCV`](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.RandomizedSearchCV.html), which will randomly samples a specified number of parameter combinations from a given range and evaluates them using cross-validation to find the best-performing combination.
The advantage of this method over other optimization (such as [`GridSearchCV`](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.GridSearchCV.html) is the computing time, which might be too long for larger datasets)

In [ ]:
for key in DecisionTreeRegressor().get_params().keys():
    print(key)

In [ ]:
hyperparameter_distributions = {
    'max_depth': np.arange(1, 20, 2).astype(int),
    'min_samples_split': np.arange(5, 105, 10).astype(int),
    'min_samples_leaf': np.arange(5, 105, 10).astype(int)
}

random_search = RandomizedSearchCV(
    dtree,
    param_distributions=hyperparameter_distributions,
    n_iter=100
)

random_search.fit(X_train, y_train.values.ravel())

The best results are

In [ ]:
print(random_search.best_params_)


**Now we can check whether the optimization of hyperparameters had any effect in the results. Plot again the comparison between test and prediction, and calculate the metrics and $\sigma_{NMAD}$ and the outlier fraction**

In [ ]:
#plot here

In [ ]:
1.48*np.median(np.abs(y_test-y_predict)/(1 + y_test))


In [ ]:
len(np.where(np.abs(y_test-y_predict)>0.15*(1+y_test))[0])/len(y_test)

**Do you think there was an improvement?**

### Part 2 Random Forest
It is clear that Decision Trees are not very good predictors of redshift. We obtain a high fraction of outliers and certainly we can do better with MSE. So, to improve the metrics, we'll try a more complex approach, with **Random Forests**, which are tree-based supervised learning models.

We initialize the model [`RandomForestRegressor()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [ ]:
model = RandomForestRegressor()

we can check the parameters of the model...notice that some of those are for each tree and others, for the forest (ensemble)

In [ ]:
model.get_params()

Now, we will use cross validation to obtain the metrics with training and test data, and to get some diagnostics of the results. Below is the implementation, check the parameters of KFold and try to change it and see what you obtain. More information about cross validation, KFold and other methods in this [link](https://scikit-learn.org/1.5/modules/cross_validation.html).

The results are the metric scores (as default, it calculates $R^2$, but you can change this in Kfold with the `scoring` parameter of cross_validate (see sklearn documentation)

In [ ]:
%%time
#it will take about 2 minutes
scores = cross_validate(model,features,target, cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score=True)

In [ ]:
scores

In [ ]:
np.mean(scores['test_score'])

In [ ]:
np.mean(scores['train_score'])

**How does the train score compares with the test score? What can you say about that?**

Let's compare the true values with the predictions. When we run `cross_validate()` we only obtain the metric scores, if we want the predictions, we use `cross_val_predict()`

In [ ]:
ypred = cross_val_predict(model,features,target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

**Plot the comparison between prediction and true values for random forests**

In [ ]:
#plot here

**Also, check and compare the distribution of true and predicted redshifts (histogram)**

In [ ]:
#plot here

**Calculate the $\sigma_{NMAD}$ and outliers fraction and again, compare with the paper...**

You probably notice that we are not doing so good. We have a **high variance**, then we are **overfitting**. This is typical for trees and forest, when are "unleashed". We can try to optimize the hyperparameters of the model. To save some time, we will select a random subsample of the data and optimize with it.

**Select a random subsample of the data (features and target)**

In [ ]:
#code here

**It is good practice to ensure that the performance on the smaller set remains similar to the one obtained on the entire data set, which means that the change in size will not significantly affect the optimization process. Check the metrics with cross_validate and compare with the one obtained previously with random forest (our benchmark)**

In [ ]:
#code here

**If the metrics are similar as before, we can move on to the optimization of hyperparameters. As in the case of Decision trees, perform a `RandomizedSearchCV(). Check also the hyperparameters of RF, and decide which ones you want to use. More information of the parameters you can find [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)**


As as suggestion, use this grid

`
parameters = {
    'min_impurity_decrease': [0.1, 0.5, 0.0],
    'max_features': [None, 4, 2],
    'n_estimators': [50, 100, 200],
    'min_samples_split': [10, 20, 100],
    'max_leaf_nodes': [None, 100, 200]
}`


In [ ]:
model.get_params()

In [ ]:
#code here

We can create a dataframe with the scores for each combination of parameters, sorted from the best models to the worst

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)


In [ ]:
scoresCV

Did we obtain better results?

Probably you already notice that we have not improve the results. :(

### Part 3: A little of Feature engineering

Before giving up, and after noting that our issue is so severe that it's hard to attribute it to an optimization or chooice of algorithm issue, we need to look more carefully at data cleaning and/or imputing (something, in fact, that should be the first step in our pipeline!)


If you check the [paper](https://arxiv.org/pdf/1903.08174) (Table 6) you will notice that the authors consider many more columns that the ones we are using here. There are in particular some columns with information about the *quality* of the data provided. These columns are
- 'subaru_source':  source of the Y -band photometry: 0 = deep pointing; 1 = shallow pointing; -99 = not observed
- 'cfhtls_source': source of the ugri(i2)z photometry: 0 = Deep field; 1 = Wide field; -99 = not observed
- 'zquality':  (In DEEP2/3 catalog) DEEP2/3 redshift quality flag

**Create a dataframe with the columns we selected before and the three new ones mentioned above**

In [ ]:
#code here

**So, we need to filter the data based on quality. Filter your data with these criterias:**

- **redshift quality: only use objects with high-quality spectroscopic redshift measurements 'zquality' >= 3**
- **select objects with cfhtls deep photometric data ('cfhtls_source' == 0)**
- **select objects with subaru deep photometry ('subaru_source' == 0)**
- **Also, unavailable measurements are marked by -99 or 99 (while typical values are around 20-25). We also get rid of data with missing measurements.**


In [ ]:
#code here


**Of course, remember to select from the target array the corresponding rows from your selections above**

In [ ]:
#code here

**Now, repeat the cross_validate() process with RandomForestRegressor (you can copy the code above) but using the selected data and obtain the metrics**

In [ ]:
#code here

**You can try to optimized the hyperparameters again to improve the results**

In [ ]:
#code here

In [ ]:
#code here

**Re plot the comparison between true and predicted values and calculate the outlier fraction and $\sigma_{NMAD}$ and compare them to the paper**

In [ ]:
#plot here

**So, what do you think? did it improve the result??**

**In any case, the lesson here is that with a simple improvement in the data (selecting by data quality) we improve much more the model than only optimizing the hyperparameters. That's the reason why is so important to explore and understand the data!**

### Part 4: Let's go neural!

We now will use  a fully connected neural network to solve the problem. With RF we already obtain a very good result, but let's see if we can improve it substantially with a more complex sophisticated model, such as NN

In [ ]:
import tensorflow as tf

In [ ]:
import keras

from keras.models import Sequential #the model is built adding layers one after the other

from keras.layers import Dense #fully connected layers: every output talks to every input

from keras.layers import Dropout #for regularization

WE will make a copy of the sel_features and sel_targets to name them X and y respectively. We will use the better quality selection of the whole data

In [ ]:
X=sel_features.copy()
y=sel_target.copy()

We will split the data in train/validation/test. First we will shuffle them

In [ ]:
X,y = shuffle(X,y, random_state = 12)

In [ ]:
fifth = int(len(y)/5) #Divide data in fifths to use 60/20/20 split

In [ ]:
X_train = X.values[:3*fifth,:]
y_train = y[:3*fifth]

X_val = X.values[3*fifth:4*fifth,:]
y_val = y[3*fifth:4*fifth]

X_test = X.values[4*fifth:,:]
y_test = y[4*fifth:]

 we need to scale our data! NNs use matrix operations with weights and biases to compute activations. These involve multiplying input features by weights and passing them through activation functions like ReLU or sigmoid. Large values of the features may affect these operations, affect the convergence of the NN and avoid a bias towards features with highr values



In [ ]:
scaler = StandardScaler()

scaler.fit(X_train)

In [ ]:
Xst_train = scaler.transform(X_train)
Xst_val = scaler.transform(X_val)
Xst_test = scaler.transform(X_test)

**In a regression problem, we will choose a different activation for the output layer (e.g. linear), and an appropriate loss function (MSE, MAE, ...).**

**Our input layer has six neurons for this problem.**

**For other parameters and the network structure, we can start with two layers with 100 neurons and go from there.**

**Try to implement the NN (Sequential) with the architecture described. if you get loss, see the solution ;). As optimizer use Adam, and the activation function for intermediate layers, use RELU**

**Train the NN with  100 epochs and batch size = 300.**

In [ ]:
results = model.evaluate(Xst_test, y_test)
print('MSE:', results) #we are only monitoring the MSE

**Plot the loss throughout the training process.**

In [ ]:
#plot here

**As always with regression problems, it is helpful to plot the predictions against the true values.**


In [ ]:
#plot here


**We didn't do cross validation, so generate prediction on our single test fold in order to derive the other metrics we are interested in (OLF and NMAD).**

In [ ]:
#code here

**Calculate Outlier Fraction**

**Calculate Normalized Median Absolute Deviation (NMAD)**

**So What was best? RF o NNs?**

**Ok, this was a long exercise... What are your conclusions ??**

### Appendix: Optimization of parameters with Keras (optional)

Below is implemented optimization with Keras tuner. It takes a long time! you can follow the cells to see hoy it works ;)

### Let's try some optimization with keras tuner

In [ ]:
 !pip3 install keras-tuner --upgrade    #You may have to install keras tuner

In [ ]:
from keras_tuner.tuners import RandomSearch
from tensorflow.keras import layers

#Some material below is adapted from the Keras Tuner documentation

# https://keras-team.github.io/keras-tuner/

This function specifies which parameters we want to tune. Tunable parameters can be of type "Choice" (we specify a set), Int, Boolean, or Float.

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 6)): #We try between 2 and 6 layers
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=100, #Each of them has 100-300 neurons, in intervals of 100
                                            max_value=300,
                                            step=100),
                               activation='relu'))
    model.add(Dense(1, activation='linear')) #last one
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), #And a few learning rates
        loss='mse')
    return model

Next, we specify how we want to explore the parameter space. The Random Search is the simplest choice, but often quite effective; alternatives are Hyperband (optimized Random Search where a larger fraction of models is trained for a smaller number of epochs, but only the most promising ones survive), or Bayesian Optimization, which attempts to build a probabilistic interpretation of the model scores (the posterior probability of obtaining score x, given the values of hyperparameters).

In [ ]:
tf.keras.backend.clear_session()

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=40, #number of combinations to try
    executions_per_trial=3,
    project_name='My Drive/Photoz') #may need to delete or reset

We can visualize the search space below:

In [ ]:
tuner.search_space_summary()

Finally, it's time to put our tuner to work. (This is a big job!)

In [ ]:
tuner.search(Xst_train, y_train, #same signature as model.fit
             epochs=100, validation_data=(Xst_val, y_val), batch_size=300, verbose = 1)

#Note: setting verbosity to 0 would give no output until done - it took about ~35 mins on my laptop

The "results\_summary(n)" function gives us access to the n best models. It's useful to look at a few because often the differences are minimal, and a smaller model might be preferable! Note that the "number of units" parameter would have a value assigned to it for each layers (even if the number of layers is smaller in that particular realization).

In [ ]:
tuner.results_summary(6)

The losses of the first few models are very similar, suggesting that 1. as usual, we need to do some form of cross-validation to be able to come up with a ranking, and 2. With 3-5 layers and a few hundred neurons per layer, the exact configuration doesn't matter too much.

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0] #choose first model

In [ ]:
best_hps.get('learning_rate')

In [ ]:
best_hps.get('num_layers')

In [ ]:
#Size of layers

print(best_hps.get('units_0'))
print(best_hps.get('units_1'))
print(best_hps.get('units_2'))

In [ ]:
model = tuner.hypermodel.build(best_hps) #define model = best model

In [ ]:
model.build(input_shape=(None,6)) #build best model (if not fit yet, this will give access to summary)

In [ ]:
model.summary()

Now, build a neural net with the optimal hyperparameters.

In [ ]:
bestnet = model.fit(Xst_train, y_train, validation_data= (Xst_val, y_val), epochs=100, batch_size=300)

We can also look at the train vs validation curves for the optimal model found by the tuner.

In [ ]:
plt.plot(bestnet.history['loss'], label = 'train')
plt.plot(bestnet.history['val_loss'],'-.m', label = 'validation')
plt.ylabel('Loss', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.ylim(0,0.1)
plt.legend(loc='upper right', fontsize = 12)
plt.legend(fontsize = 12);
#plt.savefig('OptimalNN_Photoz.png',dpi=300)

Finally, we report test scores for all the metrics of interest (MSE, OLF, NMAD):

In [ ]:
model.evaluate(Xst_test, y_test)

In [ ]:
ypred = model.predict(Xst_test)

#Calculate OLF

print('OLF', len(np.where(np.abs(y_test-ypred)>0.15*(1+y_test))[0])/len(y_test))

#Calculate Normalized Median Absolute Deviation (NMAD)

print('NMAD', 1.48*np.median(np.abs(y_test-ypred)/(1 + y_test)))

These numbers have somewhat improved, compared to the baseline model; note that whether improvement is significant should be determined via cross validation.